# End-to-End AutoML for Insurance Cross-Sell

## Part 3 - H2O AutoML with MLflow

### Contents
[Part 1 - Initial Setup](#setup)  
[Part 2 - H2O AutoML Training with MLflow Tracking](#automl)  
[Part 3 - Predict with H2O AutoML Best Model](#predict)  
[Part 4 - H2O Model Explainability](#explain)  
[Part 5 - References](#references)

___
<a name="setup"></a>
## (1) Initial Setup

### Install pre-requisite dependencies

In [24]:
!pip install requests
!pip install tabulate
!pip install future

     |████████████████████████████████| 63 kB 411 kB/s            
     |████████████████████████████████| 138 kB 840 kB/s            
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
h2o 3.34.0.3 requires tabulate, which is not installed.
docker 5.0.3 requires websocket-client>=0.32.0, which is not installed.
databricks-cli 0.16.2 requires six>=1.10.0, which is not installed.
databricks-cli 0.16.2 requires tabulate>=0.7.7, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
databricks-cli 0.16.2 requires six>=1.10.0, which is not installed.


### Install H2O in Python

In [25]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html


### Install MLflow

In [26]:
!pip install mlflow

     |████████████████████████████████| 15.6 MB 25 kB/s             
     |████████████████████████████████| 1.6 MB 37 kB/s             
     |████████████████████████████████| 1.1 MB 2.1 MB/s            
     |████████████████████████████████| 40 kB 1.3 MB/s            
  Using cached sqlparse-0.4.2-py3-none-any.whl (42 kB)
     |████████████████████████████████| 503 kB 543 kB/s            
     |████████████████████████████████| 39.8 MB 89 kB/s             
     |████████████████████████████████| 16.8 MB 212 kB/s            
     |████████████████████████████████| 661 kB 1.1 MB/s            
     |██████████████████████████▏     | 9.4 MB 813 kB/s eta 0:00:03

### Import dependencies and datasets

In [1]:
# Import libraries
import h2o
from h2o.automl import H2OAutoML, get_leaderboard

import mlflow
import mlflow.h2o
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

import pandas as pd
import json

from sklearn.metrics import f1_score, accuracy_score

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings("ignore")

___
<a name="automl"></a>
## (2) H2O AutoML Training with MLFlow Tracking

___
<a name="predict"></a>
## (3) Predict with H2O AutoML Best Model

___
<a name="explain"></a>
## (4) H2O Model Explainability
- More info: https://docs.h2o.ai/h2o/latest-stable/h2o-docs/explain.html#output-explanations